# Analysis of one example chunk that throws a value error

The chunk with the id: '201358.0_220045.0_2151-11-05 15:52:00' throws a value error for iteration with id 28 in the arima configuration (train size=12)

## Load Data

In [ ]:
# read dict that holds this chunk with all preprocessed iterations

import pandas as pd
import pmdarima as pm
import numpy as np
import time
import pyarrow as pa
import pickle

path_to_data = '../data/'

starttime = time.time()
print('Start reading the input file.')

TRAIN=12
input_file = open(str(path_to_data) + 'dict_of_chunk_steady_train_12_error_chunk_example.pickle', 'rb')
dict_of_chunk_series_with_test_and_train_error = pickle.load(input_file)
input_file.close()

endtime = round(((time.time() - starttime) / 60), 5)
print('Reading of the input file completed after '+str(endtime)+' minutes.')

In [ ]:
# Example of retrieving all iterations of this chunk:
dict_of_chunk_series_with_test_and_train_error['201358.0_220045.0_2151-11-05 15:52:00'].keys()
# As the train size = 12, the first iteration key is 11 (we choose the last index of the train set as dictionary key)

In [ ]:
# Example of accessing one iteration of this chunk
dict_of_chunk_series_with_test_and_train_error['201358.0_220045.0_2151-11-05 15:52:00'][11]
# we retrieve all Train and Tests lists for all available resampling methods (Median,Mean,Min,Max)

## Perform ARIMA for all iterations of this chunk

Will run into Value Error in iteration 28

In [ ]:
import pandas as pd
import pmdarima as pm
import numpy as np
import copy
import time
import warnings

TRAIN = 12
starttime = time.time()

# Expand the previously created dictionary (dict_of_chunk_series_with_test_and_train) to also hold the prediction series next to the train and the test series (and threshold values for test)

runningtime = round(((time.time() - starttime) / 60), 5)
print('Starting setting up dictionaries. Running time '+str(runningtime)+' min.')

dict_of_chunk_series_with_test_and_train_and_forecast = copy.deepcopy(dict_of_chunk_series_with_test_and_train_error)
dict_of_chunk_series_with_forecast_df = {}
accuracy_dict_for_chunk_iterations = {}
chunk_iterations_with_value_error = pd.DataFrame(columns=["CHUNK_ID_FILLED_TH","ITERATION","ERROR_MSG"])

np.seterr(all='ignore')


runningtime = round(((time.time() - starttime) / 60), 5)
print('Completed setting up dictionaries. Running time '+str(runningtime)+' min.')

for j, chunk in enumerate(dict_of_chunk_series_with_test_and_train_and_forecast):
    dict_of_chunk_series_with_forecast_df[chunk] = {}
    accuracy_dict_for_chunk_iterations[chunk] = {}

    runningtime = round(((time.time() - starttime) / 60), 5)
    print('CHUNK '+str(j)+': START. Running time '+str(runningtime)+' min.')
    
    for i, chunk_iteration in enumerate(dict_of_chunk_series_with_test_and_train_and_forecast[chunk]):
        
        TEST = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MEDIAN"].size
        tp, tn, fp, fn = 0, 0, 0, 0
        accurracy_matrix_df_for_chunk_iteration = pd.DataFrame(columns=["TP","FN","FP","TN"])

        ########################
        # ARIMA
        ########################

        current_train_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TRAIN_LIST_MEDIAN"] 
        current_test_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["TEST_LIST_MEDIAN"]
        
        try:
            arima = pm.auto_arima(current_train_list, seasonal = False, error_action='ignore')
            forecast_arima = pd.Series(arima.predict(TEST), index=[*range(i+TRAIN,i+TRAIN+TEST,1)], name="forecast_list_arima")
            dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["FORECAST_LIST_ARIMA"] = forecast_arima
            
            runningtime = round(((time.time() - starttime) / 60), 5)
            print('Chunk '+str(j)+' (ID: '+str(chunk)+') iteration '+str(chunk_iteration)+': Completed ARIMA. Running time '+str(runningtime)+' min.')

            # extract threshold series 
            threshold_high_for_test_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["THRESHOLD_HIGH_FOR_TEST_LIST"]
            threshold_low_for_test_list = dict_of_chunk_series_with_test_and_train_and_forecast[chunk][chunk_iteration]["THRESHOLD_LOW_FOR_TEST_LIST"]
            
            # write to dict_of_chunk_series_with_forecast_df dataframe
            all_dict_lists_as_df = pd.concat([current_test_list,threshold_high_for_test_list,threshold_low_for_test_list,forecast_arima],axis=1)
            dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration] = all_dict_lists_as_df

            ##############################################
            # Add information whether alarm was triggered
            ##############################################

            df_for_chunk_iteration = dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration]
            
            # True alarms
            df_for_chunk_iteration['high_alarm_triggered'] = np.where(df_for_chunk_iteration['test_list_median'] > df_for_chunk_iteration['threshold_high_for_test_list'] ,1,0)
            df_for_chunk_iteration['low_alarm_triggered'] = np.where(df_for_chunk_iteration['test_list_median'] < df_for_chunk_iteration['threshold_low_for_test_list'] ,1,0)
                    
            # ARIMA forecast
            df_for_chunk_iteration['high_alarm_triggered_forecast_arima'] = np.where(df_for_chunk_iteration['forecast_list_arima'] > df_for_chunk_iteration['threshold_high_for_test_list'],1,0)
            df_for_chunk_iteration['low_alarm_triggered_forecast_arima'] = np.where(df_for_chunk_iteration['forecast_list_arima'] < df_for_chunk_iteration['threshold_low_for_test_list'],1,0)
            # write to dict_of_chunk_series_with_forecast_and_alarm_df dataframe
            dict_of_chunk_series_with_forecast_df[chunk][chunk_iteration] = df_for_chunk_iteration
            
            runningtime = round(((time.time() - starttime) / 60), 5)
            print('Chunk '+str(j)+' (ID: '+str(chunk)+') iteration '+str(chunk_iteration)+': Completed Alarm Identification. Running time '+str(runningtime)+' min.')

            ##########################################
            # Calculate Confusion Matrix - High Alarms
            ##########################################

            # select true high alarms triggered
            column_index_of_high_alarm_triggered = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered")

            # select predicted high alarms
            column_index_of_high_alarm_triggered_forecast_arima = df_for_chunk_iteration.columns.get_loc("high_alarm_triggered_forecast_arima")
            
            # create df with bot as column
            high_alarms = df_for_chunk_iteration.iloc[0:,[column_index_of_high_alarm_triggered,column_index_of_high_alarm_triggered_forecast_arima]]
            
            for row_in_high_alarms in high_alarms.iterrows():

                if row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                    tp +=1
                    # print("tp", tp)
                if row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                    fn +=1
                    # print("fn", fn)
                if not row_in_high_alarms[1][0] and row_in_high_alarms[1][1]:
                    fp +=1
                    # print("fp", fp)
                if not row_in_high_alarms[1][0] and not row_in_high_alarms[1][1]:
                    tn +=1
                    # print("tn",tn)
            
            a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
            a_new_row_series = pd.Series(a_new_row,name="accuracy_high_alarms_arima")

            accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)

            runningtime = round(((time.time() - starttime) / 60), 5)
            print('Chunk '+str(j)+' (ID: '+str(chunk)+') iteration '+str(chunk_iteration)+': Completed Confusion Matrix - High Alarms. Running time '+str(runningtime)+' min.')

            #########################################
            # Calculate Confusion Matrix - Low Alarms
            ##########################################

            # Reset tp, tn, fp, fn
            tp, tn, fp, fn = 0, 0, 0, 0

            # select true low alarms triggered
            column_index_of_low_alarm_triggered = df_for_chunk_iteration.columns.get_loc("low_alarm_triggered")
            
            # select predicted low alarms
            column_index_of_low_alarm_triggered_forecast_arima = df_for_chunk_iteration.columns.get_loc("low_alarm_triggered_forecast_arima")
            
            # create df with bot as column 
            low_alarms = df_for_chunk_iteration.iloc[0:,[column_index_of_low_alarm_triggered,column_index_of_low_alarm_triggered_forecast_arima]]
            
            for row_in_low_alarms in low_alarms.iterrows():

                if row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                    tp +=1
                    # print("tp", tp)
                if row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                    fn +=1
                    # print("fn", fn)
                if not row_in_low_alarms[1][0] and row_in_low_alarms[1][1]:
                    fp +=1
                    # print("fp", fp)
                if not row_in_low_alarms[1][0] and not row_in_low_alarms[1][1]:
                    tn +=1
                    # print("tn",tn)
            
            a_new_row = {"TP":tp,"FN":fn,"FP":fp,"TN":tn}
            a_new_row_series = pd.Series(a_new_row,name="accuracy_low_alarms_arima")
            
            accurracy_matrix_df_for_chunk_iteration = accurracy_matrix_df_for_chunk_iteration.append(a_new_row_series)

            runningtime = round(((time.time() - starttime) / 60), 5)
            print('Chunk '+str(j)+' (ID: '+str(chunk)+') iteration '+str(chunk_iteration)+': Completed Confusion Matrix - Low Alarms. Running time '+str(runningtime)+' min.')
            
            # Write confusion matrix into dictionary
            accuracy_dict_for_chunk_iterations[chunk][chunk_iteration] = accurracy_matrix_df_for_chunk_iteration

        except ValueError as ve:

            ve_string = str(ve)
            a_new_row = {"CHUNK_ID_FILLED_TH":chunk,"ITERATION":chunk_iteration,"ERROR_MSG":ve_string}
            a_new_row_series = pd.Series(a_new_row)
            chunk_iterations_with_value_error = chunk_iterations_with_value_error.append(a_new_row_series, ignore_index = True)
            print("VALUE ERROR DETECTED")



    runningtime = round(((time.time() - starttime) / 60), 5)
    print('Chunk '+str(j)+' (ID: '+str(chunk)+' ) : Completed chunk. Running time '+str(runningtime)+' min.')
    print('--------------------')

endtime = round(((time.time() - starttime) / 60), 5)
print('DONE')
print('Completed in '+str(endtime)+' minutes.')


## Analyze iteration with value error independently


In [ ]:
#extract relevant train list
train_list_iteration_28= dict_of_chunk_series_with_test_and_train_error['201358.0_220045.0_2151-11-05 15:52:00'][28]["TRAIN_LIST_MEDIAN"]

In [ ]:
# perform arima model fit
arima_iteration_28 = pm.auto_arima(train_list_iteration_28, seasonal = False)

In [ ]:
#inspect arima model
arima_iteration_28

In [ ]:
# perform prediction of the next step -> leads to ValueError in _assert_all_finite(X, allow_nan, msg_dtype)
forecast_arima_iteration_28 = pd.Series(arima_iteration_28.predict(1))